In [3]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import TIFUKNNTimeDays
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

# TaFeng

Fix seed:

In [3]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [4]:
corpus_path = "./data/"
dataset_name = "ta_feng"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 32266, #items = 23812, #clicks = 817741 (#illegal records = 0)
After preprocessing: #users = 7358, #items = 11202, #clicks = 368951
Saving dataset in ./data//data_ta_feng/...


In [5]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [6]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 7358/7358 [00:15<00:00, 489.34it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:03<00:00, 2424.25it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 3026.00it/s]


In [8]:
def objective(trial):
    params = {
        "model": TIFUKNNTimeDays(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            group_size_days=trial.suggest_int("group_size_days", 1, 365),
            within_decay_rate=trial.suggest_float("within_decay_rate", 0.1, 1.0, step=0.1),
            group_decay_rate=trial.suggest_float("group_decay_rate", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.1),
            use_log = trial.suggest_categorical("use_log", [True, False]),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [10]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-29 10:05:16,973] A new study created in memory with name: no-name-edec6e64-1447-4b55-a29a-2a08c121cda2


TIFUKNNTimeDays fitting...


100%|██████████| 7358/7358 [01:07<00:00, 109.02it/s]
[I 2023-04-29 10:06:31,602] Trial 0 finished with value: 0.08417837669486034 and parameters: {'group_size_days': 282, 'within_decay_rate': 0.1, 'group_decay_rate': 0.7000000000000001, 'nearest_neighbors_num': 375, 'alpha': 0.5, 'use_log': True}. Best is trial 0 with value: 0.08417837669486034.


TIFUKNNTimeDays fitting...


100%|██████████| 7358/7358 [01:15<00:00, 97.73it/s] 
[I 2023-04-29 10:07:52,680] Trial 1 finished with value: 0.08107169520651933 and parameters: {'group_size_days': 278, 'within_decay_rate': 0.2, 'group_decay_rate': 0.1, 'nearest_neighbors_num': 350, 'alpha': 1.0, 'use_log': False}. Best is trial 0 with value: 0.08417837669486034.


...

100%|██████████| 7357/7357 [11:37<00:00, 10.55it/s]
[I 2023-04-29 12:22:34,287] Trial 22 finished with value: 0.11297487437897484 and parameters: {'group_size_days': 178, 'within_decay_rate': 1.0, 'group_decay_rate': 0.30000000000000004, 'nearest_neighbors_num': 500, 'alpha': 0.30000000000000004, 'use_log': True}. Best is trial 14 with value: 0.11312625017928453.


TIFUKNNTimeDays fitting...


100%|██████████| 7357/7357 [10:43<00:00, 11.43it/s]
[I 2023-04-29 12:34:30,339] Trial 23 finished with value: 0.1124514092509248 and parameters: {'group_size_days': 187, 'within_decay_rate': 0.9, 'group_decay_rate': 0.1, 'nearest_neighbors_num': 400, 'alpha': 0.4, 'use_log': True}. Best is trial 14 with value: 0.11312625017928453.


TIFUKNNTimeDays fitting...


100%|██████████| 7357/7357 [11:15<00:00, 10.89it/s]
[I 2023-04-29 12:47:00,249] Trial 24 finished with value: 0.10333602826948025 and parameters: {'group_size_days': 246, 'within_decay_rate': 0.7000000000000001, 'group_decay_rate': 0.30000000000000004, 'nearest_neighbors_num': 450, 'alpha': 0.30000000000000004, 'use_log': True}. Best is trial 14 with value: 0.11312625017928453.


Test:

In [11]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": TIFUKNNTimeDays(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        group_size_days=study.best_params["group_size_days"],
        within_decay_rate=study.best_params["within_decay_rate"],
        group_decay_rate=study.best_params["group_decay_rate"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        alpha=study.best_params["alpha"],
        use_log=study.best_params["use_log"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

train dataset preparing...


100%|██████████| 7358/7358 [00:15<00:00, 470.28it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 2539.10it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 3034.50it/s]


TIFUKNNTimeDays fitting...


100%|██████████| 7358/7358 [01:07<00:00, 108.31it/s]


In [12]:
trainer.evaluate(mode="test")

100%|██████████| 7357/7357 [08:30<00:00, 14.40it/s]


{'precision': 0.061519641158080736,
 'recall': 0.1503349546760062,
 'ndcg': 0.1248435198538412}

# TaoBao

Fix seed:

In [4]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 10 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [5]:
corpus_path = "./data/"
dataset_name = "taobao"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(10, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 672404, #items = 638962, #clicks = 2015807 (#illegal records = 0)
After preprocessing: #users = 10092, #items = 22286, #clicks = 67991
Saving dataset in ./data//data_taobao/...


In [6]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [7]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 10092/10092 [00:51<00:00, 196.96it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 21531.86it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 22219.24it/s]


In [8]:
def objective(trial):
    params = {
        "model": TIFUKNNTimeDays(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            group_size_days=trial.suggest_int("group_size_days", 1, 365),
            within_decay_rate=trial.suggest_float("within_decay_rate", 0.1, 1.0, step=0.1),
            group_decay_rate=trial.suggest_float("group_decay_rate", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.1),
            use_log = trial.suggest_categorical("use_log", [True, False]),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-29 19:20:27,224] A new study created in memory with name: no-name-c16507f0-75d6-4cd0-b752-d83b141dd5c3


TIFUKNNTimeDays fitting...


100%|██████████| 9307/9307 [34:45<00:00,  4.46it/s]
[I 2023-04-29 19:56:40,036] Trial 0 finished with value: 0.07412356489993777 and parameters: {'group_size_days': 282, 'within_decay_rate': 0.1, 'group_decay_rate': 0.7000000000000001, 'nearest_neighbors_num': 375, 'alpha': 0.5, 'use_log': True}. Best is trial 0 with value: 0.07412356489993777.


TIFUKNNTimeDays fitting...


100%|██████████| 9307/9307 [37:57<00:00,  4.09it/s]
[I 2023-04-29 20:36:39,638] Trial 1 finished with value: 0.07461211446161903 and parameters: {'group_size_days': 278, 'within_decay_rate': 0.2, 'group_decay_rate': 0.1, 'nearest_neighbors_num': 350, 'alpha': 1.0, 'use_log': False}. Best is trial 1 with value: 0.07461211446161903.


...

100%|██████████| 9307/9307 [34:30<00:00,  4.49it/s]
[I 2023-04-30 06:22:27,641] Trial 20 finished with value: 0.07510070831765085 and parameters: {'group_size_days': 241, 'within_decay_rate': 0.2, 'group_decay_rate': 0.5, 'nearest_neighbors_num': 450, 'alpha': 1.0, 'use_log': True}. Best is trial 17 with value: 0.07570213983766064.


TIFUKNNTimeDays fitting...


100%|██████████| 9307/9307 [36:49<00:00,  4.21it/s]
[I 2023-04-30 07:00:55,832] Trial 21 finished with value: 0.07562455779004285 and parameters: {'group_size_days': 180, 'within_decay_rate': 0.30000000000000004, 'group_decay_rate': 0.30000000000000004, 'nearest_neighbors_num': 500, 'alpha': 0.8, 'use_log': True}. Best is trial 17 with value: 0.07570213983766064.


TIFUKNNTimeDays fitting...


 39%|███▉      | 3609/9307 [14:04<26:20,  3.60it/s]

Test:

In [8]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": TIFUKNNTimeDays(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        group_size_days=study.best_params["group_size_days"],
        within_decay_rate=study.best_params["within_decay_rate"],
        group_decay_rate=study.best_params["group_decay_rate"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        alpha=study.best_params["alpha"],
        use_log=study.best_params["use_log"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

train dataset preparing...


100%|██████████| 10092/10092 [00:30<00:00, 326.20it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 39460.79it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 32211.14it/s]


TIFUKNNTimeDays fitting...


100%|██████████| 10092/10092 [00:44<00:00, 225.52it/s]


In [9]:
trainer.evaluate(mode="test")

100%|██████████| 9307/9307 [17:22<00:00,  8.93it/s]


{'precision': 0.007832814010959494,
 'recall': 0.07634038895455032,
 'ndcg': 0.05425719248855343}

# Dunnhumby

Fix seed:

In [3]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [4]:
corpus_path = "./data/"
dataset_name = "dunnhumby"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 2500, #items = 92339, #clicks = 2595370 (#illegal records = 0)
After preprocessing: #users = 2358, #items = 26756, #clicks = 1976796
Saving dataset in ./data//data_dunnhumby/...


In [5]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [6]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 2358/2358 [00:14<00:00, 167.99it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:13<00:00, 169.00it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:13<00:00, 168.47it/s]


In [7]:
def objective(trial):
    params = {
        "model": TIFUKNNTimeDays(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            group_size_days=trial.suggest_int("group_size_days", 1, 365),
            within_decay_rate=trial.suggest_float("within_decay_rate", 0.1, 1.0, step=0.1),
            group_decay_rate=trial.suggest_float("group_decay_rate", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.1),
            use_log = trial.suggest_categorical("use_log", [True, False]),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [8]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-29 13:00:14,299] A new study created in memory with name: no-name-979a2065-9e76-4da3-a9ea-e10f803b01c2


TIFUKNNTimeDays fitting...


100%|██████████| 2357/2357 [05:06<00:00,  7.68it/s]
[I 2023-04-29 13:13:15,930] Trial 0 finished with value: 0.10244781767325224 and parameters: {'group_size_days': 282, 'within_decay_rate': 0.1, 'group_decay_rate': 0.7000000000000001, 'nearest_neighbors_num': 375, 'alpha': 0.5, 'use_log': True}. Best is trial 0 with value: 0.10244781767325224.


TIFUKNNTimeDays fitting...


100%|██████████| 2357/2357 [05:04<00:00,  7.73it/s]
[I 2023-04-29 13:26:14,502] Trial 1 finished with value: 0.09752797557074781 and parameters: {'group_size_days': 278, 'within_decay_rate': 0.2, 'group_decay_rate': 0.1, 'nearest_neighbors_num': 350, 'alpha': 1.0, 'use_log': False}. Best is trial 0 with value: 0.10244781767325224.


...

100%|██████████| 2357/2357 [05:29<00:00,  7.16it/s]
[I 2023-04-29 18:06:09,611] Trial 22 finished with value: 0.15809701774436274 and parameters: {'group_size_days': 234, 'within_decay_rate': 0.9, 'group_decay_rate': 0.7000000000000001, 'nearest_neighbors_num': 350, 'alpha': 0.4, 'use_log': True}. Best is trial 17 with value: 0.16478425144345862.


TIFUKNNTimeDays fitting...


100%|██████████| 2357/2357 [05:56<00:00,  6.61it/s]
[I 2023-04-29 18:20:22,641] Trial 23 finished with value: 0.15990072490388244 and parameters: {'group_size_days': 186, 'within_decay_rate': 0.7000000000000001, 'group_decay_rate': 0.5, 'nearest_neighbors_num': 425, 'alpha': 0.30000000000000004, 'use_log': True}. Best is trial 17 with value: 0.16478425144345862.


TIFUKNNTimeDays fitting...


100%|██████████| 2357/2357 [05:35<00:00,  7.02it/s]
[I 2023-04-29 18:34:17,588] Trial 24 finished with value: 0.16399709518491712 and parameters: {'group_size_days': 36, 'within_decay_rate': 0.9, 'group_decay_rate': 0.6, 'nearest_neighbors_num': 375, 'alpha': 0.6000000000000001, 'use_log': True}. Best is trial 17 with value: 0.16478425144345862.


Test:

In [9]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": TIFUKNNTimeDays(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        group_size_days=study.best_params["group_size_days"],
        within_decay_rate=study.best_params["within_decay_rate"],
        group_decay_rate=study.best_params["group_decay_rate"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        alpha=study.best_params["alpha"],
        use_log=study.best_params["use_log"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

train dataset preparing...


100%|██████████| 2358/2358 [00:14<00:00, 159.27it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:13<00:00, 174.77it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:15<00:00, 149.19it/s]


TIFUKNNTimeDays fitting...


100%|██████████| 2358/2358 [07:50<00:00,  5.01it/s]


In [10]:
trainer.evaluate(mode="test")

100%|██████████| 2357/2357 [05:50<00:00,  6.73it/s]


{'precision': 0.1161646160373356,
 'recall': 0.17054997430939312,
 'ndcg': 0.1592542980590923}